In [1]:
import numpy as np
import pandas as pd

import conv_model as conv
import tensorflow as tf

In [2]:
#### User input ####

## path of the train and test data
train_data = "../data/digit-recognizer/train.csv"
test_data = "../data/digit-recognizer/test.csv"

## path of the test predictions
test_pred_path = "../data/digit-recognizer/test_preds1.csv"

In [3]:
## read data into Dataframe
df_train = pd.read_csv(train_data)
df_train.info()

df_test = pd.read_csv(test_data)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [4]:
## get the first 400 rows of the data
df_train = df_train.iloc[:400]
print(len(df_train.columns))

785


In [5]:
## function to create input images
def create_input_images(df):
    im_arr = []
    for row in range(0, len(df.index)):
        ## create the image frame
        image = np.zeros((28, 28, 1))

        if(row%1000 == 0):
            print(row)
        
        ## loop over the pixel values
        for col in range(1, len(df.columns)):
            image[(col-1)//28][(col-1)%28] = np.array([df.iloc[row, col]])
    
        im_arr.append(image)

    return np.array(im_arr)

In [6]:
## create the arrays of input images
im_arr = create_input_images(df_train)
#im_arr_test = create_input_images(df_test)

0


In [7]:
model1 = conv.ConvModel()

In [8]:
from tensorflow.keras.losses import CategoricalCrossentropy

In [9]:
## compile the model
model1.compile(optimizer = 'adam', loss = CategoricalCrossentropy(), metrics = ["accuracy"])

## create training input and one_hot encode
Y = df_train["label"].to_numpy()
Y_oh = tf.one_hot(Y, 10)

In [10]:
## fit the model (currently just testing the design, no final run yet)
model1.fit(im_arr, Y_oh, batch_size = 32, epochs = 5)

Epoch 1/10


2024-04-04 09:36:02.126212: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


13/13 [==============================] - 1s 6ms/step - loss: 2.5730 - accuracy: 0.2025
Epoch 2/10
13/13 [==============================] - 0s 6ms/step - loss: 1.4606 - accuracy: 0.5350
Epoch 3/10
13/13 [==============================] - 0s 6ms/step - loss: 0.7844 - accuracy: 0.7600
Epoch 4/10
13/13 [==============================] - 0s 6ms/step - loss: 0.3860 - accuracy: 0.8800
Epoch 5/10
13/13 [==============================] - 0s 5ms/step - loss: 0.2041 - accuracy: 0.9325
Epoch 6/10
13/13 [==============================] - 0s 5ms/step - loss: 0.0652 - accuracy: 0.9900
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 0.0188 - accuracy: 1.0000
Epoch 8/10
13/13 [==============================] - 0s 5ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 9/10
13/13 [==============================] - 0s 6ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/10
13/13 [==============================] - 0s 6ms/step - loss: 0.0022 - accuracy: 1.0000


In [11]:
## predict for the test data
preds = model1.predict(im_arr_test)

NameError: name 'im_arr_test' is not defined

In [ ]:
## add test predictions to the test dataframe
#print(preds.shape)
#print(len(np.argmax(preds, axis = 1)))
#print(len(np.arange(1, len(df_test.index)+1)))
#print(df_test.columns)

df_test["Label"] = np.argmax(preds, axis = 1)
df_test["ImageId"] = np.arange(1, len(df_test.index)+1)

## write results to csv file
#to_write = ["ImageId", "Label"]
#df_test.to_csv(test_pred_path, columns = to_write, index = False)